# SN and turfs

A data hacking short cut for the first presentation.  This will be refactored for sure!

In [ ]:
primst_with_counts_gdf = gpd.read_parquet('../data/processed/brooklyn/primst-turfs-counts.parq')

In [ ]:
len(primst_with_counts_gdf)

In [ ]:
primst_with_counts_gdf.PopID.is_unique

In [ ]:
primst_with_counts_popids = set(primst_with_counts_gdf.PopID)

In [ ]:
import pickle
with open('../data/processed/db/popids2.p', 'rb') as f:
    popids = pickle.load(f)

In [ ]:
popids_set = set(popids)

In [ ]:
primst_popids = primst_with_counts_popids.intersection(popids)

In [ ]:
len(primst_popids)

So, we have 49 PopIDs in the Brooklyn turfs.  We know from the previous processing they will be in both elements (PopID) and connections (Respondent PopID).

For starters I want to build a dictionary structure so I can search by primst turf and get the PopID's of _ALTER's.

In [ ]:
elements_df = pd.read_parquet('../data/processed/SN/elements.parq')
connections_df = pd.read_parquet('../data/processed/SN/connections.parq')

In [ ]:
elements_df.columns

In [ ]:
turf_connections_df = connections_df.query(f"`Respondent PopID` in {list(primst_popids)}").reset_index().drop(columns='index')

In [ ]:
turf_connections_df.groupby('Respondent PopID')['PopID _ALTER'].count()

In [ ]:
turf_connections_df['PopID _ALTER'].isnull().sum()

In [ ]:
from collections import defaultdict

In [ ]:
alternate_lookup = defaultdict(list)

In [ ]:
for i, row in turf_connections_df.iterrows():
    alternate_lookup[row['Respondent PopID']].append(row['PopID _ALTER'])

In [ ]:
len(alternate_lookup)

In [ ]:
elements_df.info()

In [ ]:
elements_df

In [ ]:
def get_count(popid):
    alternates = alternate_lookup.get(popid)
    if alternates is None:
        return 0
    else:
        return len(alternates)

In [ ]:
primst_with_counts_gdf['alter_count'] = primst_with_counts_gdf['PopID'].apply(lambda popid: get_count(popid))

In [ ]:
primst_with_counts_gdf

In [ ]:
primst_with_counts_gdf.to_parquet('../data/processed/brooklyn/primst-with-alters.parq')

In [ ]:
def org_name(popid):
    """
    Use elements_df
    """
    org = elements_df.query(f"PopID == @popid").reset_index()
    return org.iloc[0]['Label']

In [ ]:
def alters(popid):
    alter_popids = list(connections_df.query(f"`Respondent PopID` == @popid")['PopID _ALTER'])
    alter_orgs = [org_name(x) for x in alter_popids]
    return alter_orgs

In [ ]:
alters(1475)